### 문제 2-2 : 영화 리뷰 감정 분석기

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [4]:
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)


감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립


In [5]:

template = """
당신은 영화 리뷰 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [6]:
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0 
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

print("모델 및 파서 설정 완료")

모델 및 파서 설정 완료


In [7]:

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "재밌었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

테스트할 텍스트 4개 준비 완료


In [8]:
def safe_sentiment_analysis(text, use_fixing_parser=False):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        chain = prompt | model | (parser)
        result = chain.invoke({"text": text})
        return result, None
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

print("=== 영화 리뷰 감정 분석 ===")
for i, text in enumerate(texts, 1):
    result, error = safe_sentiment_analysis(text)
    if result:
        print(f"{i}. 리뷰: {text}\n   → 감정: {result.value}")
    else:
        print(f"{i}. 리뷰: {text}\n   → 오류: {error}")

=== 영화 리뷰 감정 분석 ===
1. 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   → 감정: 부정
2. 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   → 감정: 긍정
3. 리뷰: 재밌었어요!
   → 감정: 긍정
4. 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   → 감정: 중립
